In [11]:
import pandas as pd

Откройте файл Product_Sales.csv и выведите первые 10 строк.
Посмотрите на данные и скажите, что они из себя представляют.
Cколько в таблице строк и полей? Какие типы данных есть в датасете? Есть ли пустые значения?

In [ ]:
# invoice_num: Номер счета
# customer_id: ID клиента
# gender: Пол клиента
# age: Возраст клиента
# category: Категория товара
# quantity: Количество проданных товаров в рамках одного счета
# selling_price_per_unit: Стоимость продажи 1 ед.товара
# cost_price_per_unit: Себестоимость 1 ед.товара
# region: Регион
# state: Штат
# shopping_mall: Магазин

In [12]:
# вывода первых 10 строк из датафрейма Product_Sales.csv

product_sales_df = pd.read_csv('Product_Sales.csv')

# Общее количество строк и полей
total_rows = product_sales_df.shape[0]
total_columns = product_sales_df.shape[1]

# Типы данных
column_data_types = product_sales_df.dtypes

# Пустые значения
null_values = product_sales_df.isnull().sum()

print(f"Общее количество строк: {total_rows}")
print(f"Общее количество полей: {total_columns}")
print("\nТипы данных в датасете:")
print(column_data_types)
print("\nПустые значения в датасете:")
print(null_values)

Общее количество строк: 99949
Общее количество полей: 12

Типы данных в датасете:
invoice_num                object
invoice_date               object
customer_id                object
gender                     object
age                       float64
category                   object
quantity                    int64
selling_price_per_unit    float64
cost_price_per_unit       float64
region                     object
state                      object
shopping_mall              object
dtype: object

Пустые значения в датасете:
invoice_num               0
invoice_date              0
customer_id               0
gender                    6
age                       6
category                  0
quantity                  0
selling_price_per_unit    0
cost_price_per_unit       0
region                    0
state                     0
shopping_mall             0
dtype: int64


Выведите список уникальных магазинов

In [13]:
# Получения списка уникальных магазинов

unique_shopping_malls = product_sales_df['shopping_mall'].unique()
print(unique_shopping_malls)

['Kanyon' 'Viaport Outlet' 'Metrocity' 'Metropol AVM' 'Istinye Park'
 'Mall of Istanbul' 'Zorlu Center']


Посчитайте количество проданных товаров в разбивке по категориям. Выведите топ-3 наиболее продаваемых категорий товаров

In [14]:
# Подсчет количества проданных товаров по категориям
sales_by_category = product_sales_df.groupby('category')['quantity'].sum()

# Сортировка категорий по количеству проданных товаров в убывающем порядке и получение топ-3
top_3_categories = sales_by_category.sort_values(ascending=False).head(3)

# Вывод результатов
print(top_3_categories)

category
Clothing           104110
Cosmetics           45802
Food & Beverage     44512
Name: quantity, dtype: int64


Посчитайте сумму продаж по штатам. Какой штат продает больше всего, а какой - меньше?

In [27]:
# Подсчет суммы продаж по штатам
sales_by_state = product_sales_df.groupby('state')['selling_price_per_unit'].sum()

# Нахождение штата с максимальной суммой продаж
state_max_sales = sales_by_state.idxmax()
max_sales = sales_by_state.max()

# Нахождение штата с минимальной суммой продаж
state_min_sales = sales_by_state.idxmin()
min_sales = sales_by_state.min()

# Вывод результатов
print(f"Штат с максимальной суммой продаж: {state_max_sales} ({max_sales:.2f})")
print(f"Штат с минимальной суммой продаж: {state_min_sales} ({min_sales:.2f})")

Штат с максимальной суммой продаж: California (13097040.15)
Штат с минимальной суммой продаж: Indiana (939183.52)


Разбейте покупателей на возрастные группы. Группы определите самостоятельно. Какая из этих групп тратит больше всего денег на одежду?

In [23]:
# Определение возрастных групп
bins = [0, 18, 30, 40, 50, 60, 70, 100]
labels = ['0-18', '19-30', '31-40', '41-50', '51-60', '61-70', '71-100']
# делим покупателей на заданные возрастные группы с помощью функции pd.cut()
product_sales_df['age_group'] = pd.cut(product_sales_df['age'], bins=bins, labels=labels, right=False)

# Фильтрация данных по категории 'Clothing' и подсчет суммы продаж по возрастным группам
clothing_sales_by_age_group = product_sales_df[product_sales_df['category'] == 'Clothing'].groupby('age_group')['selling_price_per_unit'].sum()

# Нахождение возрастной группы с максимальной суммой продаж одежды
max_clothing_sales_age_group = clothing_sales_by_age_group.idxmax()
max_clothing_sales = clothing_sales_by_age_group.max()

print(f"Возрастная группа, тратящая больше всего денег на одежду: {max_clothing_sales_age_group}, с суммой продаж: {max_clothing_sales}")

Возрастная группа, тратящая больше всего денег на одежду: 19-30, с суммой продаж: 7261635.92


Какая доля всех покупок оплачивалась наличными? Используйте файл Product_Payment.csv, чтобы подтянуть метод оплаты

In [28]:
# Загрузка файла Product_Payment.csv
product_payment_df = pd.read_csv('Product_Payment.csv')

# Объединение данных о продажах с данными о способе оплаты по номеру счета
merged_df = product_sales_df.merge(product_payment_df, on='invoice_num')

# Подсчет количества покупок, оплаченных наличными
cash_payments_count = merged_df[merged_df['payment_method'] == 'Cash']['quantity'].sum()

# Подсчет общего количества покупок
total_purchases_count = merged_df['quantity'].sum()

# Расчет доли покупок, оплаченных наличными
cash_payment_share = cash_payments_count / total_purchases_count

# Вывод результата
print(f"Доля покупок, оплаченных наличными: {cash_payment_share:.2%}")

Доля покупок, оплаченных наличными: 44.64%
